In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim
import torch.nn.utils.prune as prune

from copy import deepcopy

In [2]:
torch.manual_seed(0);

In [3]:
from sparse_linear import *

In [4]:
def dense_to_sparse(dense_tensor: torch.Tensor) -> torch.sparse.FloatTensor:
    indices = dense_tensor.nonzero(as_tuple=True)
    values = dense_tensor[indices]
    indices = torch.stack(indices)

    sparse_tensor = torch.sparse.FloatTensor(indices, values, dense_tensor.size())
    return sparse_tensor

In [5]:
def convert_dense_to_sparse_network(model: nn.Module) -> nn.Module:
    """
    Converts a given dense neural network model to a sparse neural network model.

    This function recursively iterate through the given model and replaces all instances of 
    `nn.Linear` layers with `SparseLinear` layers

    Args:
        model (nn.Module): The dense neural network model to be converted.

    Returns:
        nn.Module: A new neural network model with sparse layers.
    """
    new_model = model.__class__()

    for name, module in model.named_children():
        if isinstance(module, nn.Linear):
            sparse_weight = dense_to_sparse(module.weight.data)
            sparse_bias = dense_to_sparse(module.bias.data)

            setattr(new_model, name, SparseLinear(sparse_weight, sparse_bias))
        else:
            setattr(new_model, name, convert_dense_to_sparse_network(module))
    return new_model

In [6]:
class SimpleFCN(nn.Module):
    def __init__(self, input_size=8):
        super(SimpleFCN, self).__init__()
        self.fc1 = nn.Linear(input_size, 4)
        self.fc2 = nn.Linear(4, 4)
        self.fc3 = nn.Linear(2, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = SimpleFCN()

In [7]:
sparse_model = convert_dense_to_sparse_network(model)

C:\Users\fedor\AppData\Local\Temp\ipykernel_11816\1220358512.py:6: UserWarning: torch.sparse.SparseTensor(indices, values, shape, *, device=) is deprecated.  Please use torch.sparse_coo_tensor(indices, values, shape, dtype=, device=). (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:653.)
  sparse_tensor = torch.sparse.FloatTensor(indices, values, dense_tensor.size())


In [8]:
linear = nn.Linear(8, 1)

sparse_weight = dense_to_sparse(linear.weight.data)
sparse_bias = dense_to_sparse(linear.bias.data)
just_sparse_linear = SparseLinear(sparse_weight, sparse_bias)

sparse_linear = deepcopy(just_sparse_linear)

sparse_putting_linear1 = SparseRecursiveLinear(sparse_linear, None)
print(sparse_putting_linear1.sparse_linear.weight_indices, "\n")
sparse_putting_linear1.replace(0, 6)
sparse_putting_linear1.replace(0, 7)
# sparse_putting_linear1.replace(0, 6)
print(sparse_putting_linear1.sparse_linear.weight_indices)
print(sparse_putting_linear1.embed_weight_indeces, "\n")

sparse_putting_linear2 = SparseRecursiveLinear(sparse_linear, sparse_putting_linear1)
sparse_putting_linear2.replace(0, 8 )
sparse_putting_linear2.replace(0, 9)
print(sparse_putting_linear1.sparse_linear.weight_indices)
print(sparse_putting_linear2.embed_weight_indeces, "\n")

sparse_putting_linear3 = SparseRecursiveLinear(sparse_linear, sparse_putting_linear2, is_last=True)
sparse_putting_linear3.replace(0, 3)
print(sparse_putting_linear1.sparse_linear.weight_indices)
print(sparse_putting_linear3.embed_weight_indeces, "\n")


tensor([[0, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 2, 3, 4, 5, 6, 7]]) 

tensor([[0, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 2, 3, 4, 5, 8, 9]])
tensor([[0, 1],
        [6, 7]]) 

tensor([[ 0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  1,  2,  3,  4,  5, 10, 11]])
tensor([[0, 1],
        [8, 9]]) 

tensor([[ 0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  1,  2,  4,  5, 10, 11, 12]])
tensor([[0],
        [3]]) 



In [9]:
from nonlinearity_metrics import *

In [10]:
criterion = nn.MSELoss()

metrics = [
    GradientMeanEdgeMetric(criterion),
    PerturbationSensitivityEdgeMetric(criterion),
]

In [11]:
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_regression

X, y = make_regression(n_samples=1000, n_features=8, n_informative=8, random_state=42)
X = torch.from_numpy(X).float()
y = torch.from_numpy(y).float()

dataset = list(zip(X, y))
train_dataset, test_dataset = train_test_split(dataset, test_size=0.2, random_state=42)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [12]:
def calculate_edge_metric_for_dataloader(model, dataloader, edgeMetric: NonlinearityMetric):
    accumulated_grads = None
    print(type(edgeMetric))
    for data, target in dataloader:
        data, target = data.to('cpu'), target.to('cpu')
        # print(data[0].shape)
        # y_pred = model(data[0])
        metric = edgeMetric.calculate(model, data, target)

        if accumulated_grads is None:
            accumulated_grads = torch.zeros_like(metric).to('cpu')

        accumulated_grads += metric

    return accumulated_grads / len(dataloader)

In [13]:
for i in metrics:
    print(calculate_edge_metric_for_dataloader(sparse_putting_linear3, test_loader, i))

<class 'nonlinearity_metrics.GradientMeanEdgeMetric'>
tensor([212.1626,  97.8102,  80.0607,  74.0020,  52.0682,   9.3702,  61.2353,
         89.4596])
<class 'nonlinearity_metrics.PerturbationSensitivityEdgeMetric'>
tensor([0.0084, 0.0084, 0.0081, 0.0077, 0.0081, 0.0005, 0.0025, 0.0039])


1 and 4 values is same because we change only edges, which go to 2 and 3 output

In [14]:
torch.tensor([[ 0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  1,  2,  4,  5, 10, 11, 12]])[1]

tensor([ 0,  1,  2,  4,  5, 10, 11, 12])

In [15]:
x = torch.rand(1, 8)
sparse_model.fc1(x)

tensor([[-0.0838, -0.5050,  0.2659, -0.3348]], grad_fn=<AsStridedBackward0>)

In [16]:
sparse_putting_linear3(x)

tensor([[0.2641]], grad_fn=<AsStridedBackward0>)

In [17]:
import torch
import torch.nn as nn
from torchviz import make_dot
sample_input = torch.randn(1, 8)

putted_output = sparse_putting_linear3(sample_input)
simple_output = sparse_model.fc1(sample_input)

putted_graph = make_dot(putted_output, params=dict(sparse_putting_linear3.named_parameters()))
simple_graph = make_dot(simple_output, params=dict(sparse_model.fc1.named_parameters()))

In [17]:
putted_graph

ExecutableNotFound: failed to execute WindowsPath('dot'), make sure the Graphviz executables are on your systems' PATH

In [18]:
simple_graph

ExecutableNotFound: failed to execute WindowsPath('dot'), make sure the Graphviz executables are on your systems' PATH